In [1]:
import re
import cupy as cp
import numpy as np
from pyqcu import define
from pyqcu import io
from pyqcu import qcu

print('My rank is ', define.rank)

params = np.array([0]*define._PARAMS_SIZE_, dtype=np.int32)
params[define._LAT_X_] = 4
params[define._LAT_Y_] = 4
params[define._LAT_Z_] = 4
params[define._LAT_T_] = 1
params[define._LAT_XYZT_] = params[define._LAT_X_] * \
    params[define._LAT_Y_] * \
    params[define._LAT_Z_] * params[define._LAT_T_]
params[define._GRID_X_] = 1
params[define._GRID_Y_] = 1
params[define._GRID_Z_] = 1
params[define._GRID_T_] = 1
params[define._PARITY_] = 0
params[define._NODE_RANK_] = 0
params[define._NODE_SIZE_] = 1
params[define._DAGGER_] = 0
params[define._MAX_ITER_] = 1e3
params[define._DATA_TYPE_] = 0
params[define._SET_INDEX_] = 2
params[define._SET_PLAN_] = -1
argv = np.array([0.0]*define._ARGV_SIZE_, dtype=np.float32)
argv[define._MASS_] = 0.0
argv[define._TOL_] = 1e-9
print("Parameters:", params)
print("Arguments:", argv)



    @@@@@@######QCU NOTES START######@@@@@@@
    1. The libqcu.so was compiled when pyqcu setup in download_path/PyQCU/lib, please add this path to your LD_LIBRARY_PATH.
    2. The QCU(PyQCU) splite grid by x->y->z->t, lattice by x->y->z->t->p->d->c->c or x->y->z->t->c->s(->p) and x->y->z->t->c->s->c->s(->p).
    3. The QUDA(PyQUDA) splite grid by t->z->y->x, lattice by c->c->x->y->z->t->p->d or c->s->x->y->z->t(->p) and c->s->c->s->x->y->z->t(->p).
    4. The QCU input params in numpy array(dtype=np.int32), argv in  numpy array(dtype=np.float32 or float64) array, set_ptrs in numpy array(dtype=np.int64), other in cupy array(dtype=cp.complex64 or complex128).
     @@@@@@######QCU NOTES END######@@@@@@@
    
My rank is  0
Parameters: [   4    4    4    1   64    1    1    1    1    0    0    1    0 1000
    0    2   -1]
Arguments: [0.e+00 1.e-09]


In [2]:
#############################
laplacian_out = cp.zeros(
    shape=params[define._LAT_XYZT_]*define._LAT_C_, dtype=cp.complex64)
print("Laplacian out:", laplacian_out)
print("Laplacian out data:", laplacian_out.data)
print("Laplacian out shape:", laplacian_out.shape)
print("norm of Laplacian out:", cp.linalg.norm(laplacian_out))
laplacian_in = cp.array([range(params[define._LAT_XYZT_]*define._LAT_C_)])
laplacian_in = cp.array(laplacian_in+1j*laplacian_in, dtype=cp.complex64)
print("Laplacian in:", laplacian_in)
print("Laplacian in data:", laplacian_in.data)
print("Laplacian in shape:", laplacian_in.shape)
print("norm of Laplacian in:", cp.linalg.norm(laplacian_in))
laplacian_gauge = cp.array(
    [range(params[define._LAT_XYZT_]*define._LAT_DCC_)])
laplacian_gauge = cp.array(
    laplacian_gauge+1j*laplacian_gauge, dtype=cp.complex64)
print("Laplacian gauge:", laplacian_gauge)
print("Laplacian gauge data:", laplacian_gauge.data)
print("Laplacian gauge shape:", laplacian_gauge.shape)
print("norm of Laplacian gauge:", cp.linalg.norm(laplacian_gauge))


Laplacian out: [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.

In [3]:
#############################
set_ptrs = np.array(params, dtype=np.int64)
print("Set pointers:", set_ptrs)
print("Set pointers data:", set_ptrs.data)
qcu.applyInitQcu(set_ptrs, params, argv)
qcu.applyLaplacianQcu(laplacian_out, laplacian_in,
                      laplacian_gauge, set_ptrs, params)
print("Laplacian out:", laplacian_out)
print("Laplacian out data:", laplacian_out.data)
print("Laplacian out shape:", laplacian_out.shape)
print("norm of Laplacian out:", cp.linalg.norm(laplacian_out))
# _=io.laplacian2czyx(laplacian_out, params)
# for x in range(params[define._LAT_X_]):
#     for y in range(params[define._LAT_Y_]):
#         for z in range(params[define._LAT_Z_]):
#             for c in range(define._LAT_C_):
#                 # print(f"({x}, {y}, {z}, {c}):", _[z, y, x, c])
#                 print(f"({c}, {x}, {y}, {z}):", _[c, x, y, z])
qcu.applyEndQcu(set_ptrs, params)


Set pointers: [   4    4    4    1   64    1    1    1    1    0    0    1    0 1000
    0    2   -1]
Set pointers data: <memory at 0x7f38e56e2080>
Laplacian out: just for laplacian, lat_t = 1, no even-odd
gridDim.x               :4
blockDim.x              :16
host_params[_LAT_X_]    :4
host_params[_LAT_Y_]    :4
host_params[_LAT_Z_]    :4
host_params[_LAT_T_]    :1
host_params[_LAT_XYZT_] :64
host_params[_GRID_X_]   :1
host_params[_GRID_Y_]   :1
host_params[_GRID_Z_]   :1
host_params[_GRID_T_]   :1
host_params[_PARITY_]   :0
host_params[_NODE_RANK_]:0
host_params[_NODE_SIZE_]:1
host_params[_DAGGER_]   :0
host_params[_MAX_ITER_] :1000
host_params[_SET_INDEX_]:2
host_params[_SET_PLAN_] :-1
host_argv[_MASS_]       :0.000000e+00
host_argv[_TOL_]        :1.000000e-09
lat_2dim[_XY_]          :16
lat_2dim[_XZ_]          :16
lat_2dim[_XT_]          :4
lat_2dim[_YZ_]          :16
lat_2dim[_YT_]          :4
lat_2dim[_ZT_]          :4
lat_3dim[_YZT_]         :16
lat_3dim[_XZT_]         :16
lat_3

In [4]:
#############################
pyquda_laplacian_gauge = io.ccdzyx2dzyxcc(io.laplacian_gauge2ccdzyx(
    laplacian_gauge, params))
pyquda_laplacian_in = io.czyx2zyxc(io.laplacian2czyx(
    laplacian_in, params))
print("PyQuda Laplacian gauge:", pyquda_laplacian_gauge)
print("PyQuda Laplacian gauge data:", pyquda_laplacian_gauge.data)
print("PyQuda Laplacian gauge shape:", pyquda_laplacian_gauge.shape)
print("norm of PyQuda Laplacian gauge:", cp.linalg.norm(pyquda_laplacian_gauge))
print("PyQuda Laplacian in:", pyquda_laplacian_in)
print("PyQuda Laplacian in data:", pyquda_laplacian_in.data)
print("PyQuda Laplacian in shape:", pyquda_laplacian_in.shape)
print("norm of PyQuda Laplacian in:", cp.linalg.norm(pyquda_laplacian_in))


PyQuda Laplacian gauge: [[[[[[0.000e+00+0.000e+00j 2.560e+02+2.560e+02j 5.120e+02+5.120e+02j]
     [7.680e+02+7.680e+02j 1.024e+03+1.024e+03j 1.280e+03+1.280e+03j]
     [1.536e+03+1.536e+03j 1.792e+03+1.792e+03j 2.048e+03+2.048e+03j]]

    [[1.000e+00+1.000e+00j 2.570e+02+2.570e+02j 5.130e+02+5.130e+02j]
     [7.690e+02+7.690e+02j 1.025e+03+1.025e+03j 1.281e+03+1.281e+03j]
     [1.537e+03+1.537e+03j 1.793e+03+1.793e+03j 2.049e+03+2.049e+03j]]

    [[2.000e+00+2.000e+00j 2.580e+02+2.580e+02j 5.140e+02+5.140e+02j]
     [7.700e+02+7.700e+02j 1.026e+03+1.026e+03j 1.282e+03+1.282e+03j]
     [1.538e+03+1.538e+03j 1.794e+03+1.794e+03j 2.050e+03+2.050e+03j]]

    [[3.000e+00+3.000e+00j 2.590e+02+2.590e+02j 5.150e+02+5.150e+02j]
     [7.710e+02+7.710e+02j 1.027e+03+1.027e+03j 1.283e+03+1.283e+03j]
     [1.539e+03+1.539e+03j 1.795e+03+1.795e+03j 2.051e+03+2.051e+03j]]]


   [[[4.000e+00+4.000e+00j 2.600e+02+2.600e+02j 5.160e+02+5.160e+02j]
     [7.720e+02+7.720e+02j 1.028e+03+1.028e+03j 1.284e+0

In [5]:

from opt_einsum import contract
def pyquda_Laplacian(F, U):
    Lx, Ly, Lz, Lt = params[define._LAT_X_], params[define._LAT_Y_], params[define._LAT_Z_], params[define._LAT_T_]
    U_dag = U.transpose(0, 1, 2, 3, 5, 4).conj()
    F = F.reshape(Lz, Ly, Lx, define._LAT_C_, -1)
    return (
        # - for SA with evals , + for LA with (12 - evals)
        6 * F
        - (
            contract("zyxab,zyxbc->zyxac", U[0], cp.roll(F, -1, 2))
            + contract("zyxab,zyxbc->zyxac", U[1], cp.roll(F, -1, 1))
            + contract("zyxab,zyxbc->zyxac", U[2], cp.roll(F, -1, 0))
            + cp.roll(contract("zyxab,zyxbc->zyxac", U_dag[0], F), 1, 2)
            + cp.roll(contract("zyxab,zyxbc->zyxac", U_dag[1], F), 1, 1)
            + cp.roll(contract("zyxab,zyxbc->zyxac", U_dag[2], F), 1, 0)
        )
    ).reshape(Lz * Ly * Lx * define._LAT_C_, -1)
    


In [6]:
pyquda_Laplacian_out = pyquda_Laplacian(
    pyquda_laplacian_in, pyquda_laplacian_gauge)
_ = io.zyxc2czyx(
    io.laplacian2zyxc(pyquda_Laplacian_out, params))
print("PyQuda Laplacian out[0]:", _[0])
print("PyQuda Laplacian out[0] data:", _[0].data)
print("PyQuda Laplacian out[0] shape:", _[0].shape)
print("norm of PyQuda Laplacian out[0]:", cp.linalg.norm(_[0]))
_ = io.array2xxx(_)
print("PyQuda Laplacian out:", _)
print("PyQuda Laplacian out data:", _.data)
print("PyQuda Laplacian out shape:", _.shape)
print("norm of PyQuda Laplacian out:", cp.linalg.norm(_))


PyQuda Laplacian out[0]: [[[-1918998.-611328.j -1915830.-618378.j -1932714.-625464.j
   -1949634.-626370.j]
  [-1899246.-639744.j -1895838.-646938.j -1912674.-654168.j
   -1929546.-655122.j]
  [-1966854.-668736.j -1963398.-676074.j -1980378.-683448.j
   -1997394.-684450.j]
  [-2035038.-666432.j -2031534.-673818.j -2048658.-681240.j
   -2065818.-682194.j]]

 [[-1800438.-728448.j -1796310.-736074.j -1813002.-743736.j
   -1829730.-744834.j]
  [-1776846.-759168.j -1772478.-766938.j -1789122.-774744.j
   -1805802.-775890.j]
  [-1843686.-790464.j -1839270.-798378.j -1856058.-806328.j
   -1872882.-807522.j]
  [-1911102.-788928.j -1906638.-796890.j -1923570.-804888.j
   -1940538.-806034.j]]

 [[-2072022.-854784.j -2067702.-862986.j -2084970.-871224.j
   -2102274.-872514.j]
  [-2047662.-887808.j -2043102.-896154.j -2060322.-904536.j
   -2077578.-905874.j]
  [-2116806.-921408.j -2112198.-929898.j -2129562.-938424.j
   -2146962.-939810.j]
  [-2186526.-920640.j -2181870.-929178.j -2199378.-937752.